# Deep Learning for Business Applications course

## TOPIC 2: Introduction to Computer Vision. Image processing with OpenCV

### 1. Library installation

Documentation for use of OpenCV with Python API [see here](https://docs.opencv.org/).

In [ ]:
!pip3 install opencv-python

In [ ]:
import os
import cv2
import numpy as np
from matplotlib import pyplot as plt

### 2. Video processing

Task came from work in [Russian Football Union](https://rfs.ru/).

In [ ]:
!ls -la /home/jovyan/__DATA/DLBA_F24/topic_02/

In [ ]:
vid_path = '/home/jovyan/__DATA/DLBA_F24/topic_02/videoplayback.mp4'

In [ ]:
# open the video from the file

cap = cv2.VideoCapture(vid_path)
frames_cnt = cap.get(cv2.CAP_PROP_FRAME_COUNT)
fps = cap.get(cv2.CAP_PROP_FPS)
print('video has {} frames and rate {} fps (frames-per-second)'.format(
    frames_cnt,
    fps
))

In [ ]:
def get_frames(vid_path, start_time, num_frames, save_dir):
    """
    Function takes the path to video
    and saves few frames to the disk.

    :vid_path: path to video file
    :start_time: where to start capturing frames
    :num_frames: ho many frames to save
    :save_dir: path to save to

    """
    cap = cv2.VideoCapture(vid_path)
    frames_cnt = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    fps = cap.get(cv2.CAP_PROP_FPS)
    start_pos = int(start_time * fps)
    end_pos = int(start_pos + num_frames)
    if end_pos <= frames_cnt:
        for frame_num in range(start_pos, end_pos):
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
            res, frame = cap.read()
            if res:
                file_name = '{}/frame_{}.png'.format(save_dir, frame_num)
                cv2.imwrite(file_name, frame)
    else:
        print('out of video lenght')

In [ ]:
!mkdir -p football

In [ ]:
START_FRAME = 10
FRAMES_TO_PROC = 3
imgs_dir = 'football'
get_frames(
    vid_path,
    START_FRAME,
    FRAMES_TO_PROC,
    imgs_dir
)

In [ ]:
!ls -la $imgs_dir

In [ ]:
os.listdir(imgs_dir)

### 3. Image processing

#### 3.1. Open an image

In [ ]:
file_path = f'{imgs_dir}/{os.listdir(imgs_dir)[0]}'
file_path

In [ ]:
img = cv2.imread(file_path)
assert img is not None, 'file could not be read, check if file exists'

In [ ]:
img

In [ ]:
type(img)

In [ ]:
img.shape

In [ ]:
# let's see our image

plt.figure(figsize=(16, 8))
plt.imshow(img)
plt.show()

In [ ]:
# RGBimage = cv2.cvtColor(BGRimage, cv2.COLOR_BGR2RGB)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(16, 8))
plt.imshow(img)
plt.show()

In [ ]:
# access to one pixel
px = img[100, 100]
print(px)

In [ ]:
# accessing only blue pixel
blue = img[100, 100, 0]
print(blue)

#### 3.2. Basic operations

In [ ]:
# draw a box around player

cv2.rectangle(
    img,
    (60, 800),
    (180, 990),
    (0, 255, 0),
    2
)
plt.imshow(img)
plt.show()

In [ ]:
# put some text on the image

cv2.putText(
    img,
    'player',
    (60, 750),
    cv2.FONT_HERSHEY_SIMPLEX,
    2,
    (0, 255, 0),
    3
)
plt.imshow(img)
plt.show()

In [ ]:
# get certain regions of images

player = img[800:990, 60:180]
plt.imshow(player)
plt.show()

In [ ]:
# image resize

height, width = player.shape[:2]
result = cv2.resize(
    player,
    (2 * width, 2 * height),
    interpolation=cv2.INTER_CUBIC
)
plt.imshow(result)
plt.show()

In [ ]:
# affine transformation
# all parallel lines in the original image
# will still be parallel in the output image
# we need three points from the input image
# and their corresponding locations in the output image

rows, cols, ch = player.shape
pts1 = np.float32([
    [25, 25],
    [75, 25],
    [25, 50]
])
pts2 = np.float32([
    [10, 40],
    [50, 50],
    [15, 75]
])

M = cv2.getAffineTransform(pts1, pts2)
dst = cv2.warpAffine(player, M, (cols, rows))

plt.subplot(121)
plt.imshow(player)
plt.title('Input')
plt.subplot(122)
plt.imshow(dst)
plt.title('Output')
plt.show()

In [ ]:
# color histogram for the image

color = ('b', 'g', 'r')
for i, col in enumerate(color):
    histr = cv2.calcHist(
        [player],
        [i],
        None,
        [256],
        [0, 256]
    )
    plt.plot(histr, color=col)
    plt.xlim([0, 256])
plt.show()

#### 3.3. Advanced

In [ ]:
# Canny Edge detection

edges = cv2.Canny(player, 100, 200)
plt.imshow(edges)
plt.show()

In [ ]:
# Image Segmentation

gray = cv2.cvtColor(player, cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(
    gray,
    0,
    255,
    cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU
)
plt.imshow(thresh)
plt.show()

In [ ]:
# Interactive foreground extraction using GrabCut algorithm
# https://docs.opencv.org/4.x/d8/d83/tutorial_py_grabcut.html

mask = np.zeros(player.shape[:2], np.uint8)

bgdModel = np.zeros((1, 65), np.float64)
fgdModel = np.zeros((1, 65), np.float64)

rect = (20, 15, 90, 160)
cv2.grabCut(player, mask, rect, bgdModel, fgdModel, 5, cv2.GC_INIT_WITH_RECT)

mask2 = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')
player_fg = player * mask2[:, :, np.newaxis]

plt.imshow(player_fg)
plt.show()

#### 3.4. Finally

In [ ]:
# Template matching
# https://docs.opencv.org/4.x/d4/dc6/tutorial_py_template_matching.html

img_gr = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img2 = img_gr.copy()

# our template will be the player
# take it as part of the image
template = cv2.cvtColor(player, cv2.COLOR_BGR2GRAY)
w, h = template.shape[::-1]

# All the 6 methods for comparison in a list
methods = [
    'TM_CCOEFF',
    'TM_CCOEFF_NORMED',
    'TM_CCORR',
    'TM_CCORR_NORMED',
    'TM_SQDIFF',
    'TM_SQDIFF_NORMED'
]

for meth in methods:
    img_ = img2.copy()
    method = getattr(cv2, meth)

    # Apply template Matching
    res = cv2.matchTemplate(img_, template, method)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)

    # If the method is TM_SQDIFF or TM_SQDIFF_NORMED, take minimum
    if method in [cv2.TM_SQDIFF, cv2.TM_SQDIFF_NORMED]:
        top_left = min_loc
    else:
        top_left = max_loc
    bottom_right = (top_left[0] + w, top_left[1] + h)

    cv2.rectangle(img_, top_left, bottom_right, 255, 8)

    plt.figure(figsize=(16, 5))
    plt.subplot(121)
    plt.imshow(res, cmap='gray')
    plt.title('Matching Result')
    plt.xticks([])
    plt.yticks([])
    plt.subplot(122)
    plt.imshow(img_, cmap='gray')
    plt.title('Detected Point')
    plt.xticks([])
    plt.yticks([])
    plt.suptitle(meth)
    plt.show()

In [ ]:
# Template matching with multiple objects

img_rgb = img.copy()
res = cv2.matchTemplate(img_, template, cv2.TM_CCOEFF_NORMED)
# try to play with the threshold e.g. set it to .3
threshold = .5
loc = np.where(res >= threshold)
for pt in zip(*loc[::-1]):
    cv2.rectangle(
        img_rgb,
        pt,
        (pt[0] + w, pt[1] + h),
        (255, 0, 0),
        2
    )

plt.figure(figsize=(16, 8))
plt.imshow(img_rgb)
plt.show()